In [1]:
import sys
import itertools
import xarray as xr
import numpy as np
import pandas as pd
%matplotlib inline
# import wrf


# import personal modules
# Path to modules
sys.path.append('../modules')
# Import my modules
from utils import select_months_ds
from statistical_tests import ttest_1samp_new

In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
## load ar dates within region
region = 'gulf_of_mexico' ## 'san_juan' 'baja' 'gulf_of_mexico'
fname = '../out/bbox_dates_{0}.csv'.format(region)
df = pd.read_csv(fname)
df['day'] = pd.to_datetime(df['datetime']).dt.normalize()
df = df.sort_values(by=['datetime'])
# df = df.set_index(pd.to_datetime(df['datetime'])) ## set daily values as index
ar_dates = df['day'].values
## create year month columns in dataframe 
# df['yearmonth'] = df.index.strftime("%Y%m")
new = df.drop_duplicates('day')
ar_dates = new['day'].values

In [4]:
new

,Unnamed: 0,datetime,day
667,6,2000-04-27 20:00:00,2000-04-27
663,2,2000-04-28 00:00:00,2000-04-28
1343,2,2000-08-16 04:00:00,2000-08-16
2214,9,2000-10-21 16:00:00,2000-10-21
1344,8,2000-10-22 00:00:00,2000-10-22
...,...,...,...
2200,15,2023-08-22 08:00:00,2023-08-22
1763,234,2023-09-30 15:00:00,2023-09-30
1573,265,2023-10-01 00:00:00,2023-10-01
643,319,2023-10-02 00:00:00,2023-10-02


In [5]:
def compute_horizontal_composites(varname, anomaly, ar_dates, ssn, region):
    ## function that computes composites (anomaly or non-anomaly) for defined start_mon and end_mon    
    ## for DJF, MAM, JJA, SON, NDJFMA, and MJJASON
    ## compute anomaly composites - anomaly = True
    ## compute non-anomaly composites - anomaly=False

    ## set start_mon and end_mon based on ssn
    if ssn == 'DJF':
        start_mon, end_mon = (12, 2)
    elif ssn == 'MAM':
        start_mon, end_mon = (3, 5)
    elif ssn == 'JJA':
        start_mon, end_mon = (6, 8)
    elif ssn == 'SON':
        start_mon, end_mon = (9, 11)
    elif ssn == 'NDJFMA':
        start_mon, end_mon = (11, 4)
    elif ssn == 'MJJASO':
        start_mon, end_mon = (5, 10)

    ## load data
    path_to_data = '/expanse/nfs/cw3e/cwp140/downloads/ERA5/'
    out_path = '/home/dnash/DATA/preprocessed/ERA5_composites/'
    
    if anomaly == True:
        fname_pattern = path_to_data + '{0}/anomalies/daily_filtered_anomalies_{0}_*.nc'.format(varname)
    else:
        fname_pattern = path_to_data + '{0}/daily/era5_namerica_025dg_daily_{0}_*.nc'.format(varname)

    ds = xr.open_mfdataset(fname_pattern, engine='netcdf4', combine='by_coords')

    ## subset to AR dates
    ds = ds.sel(time=ar_dates)

    ## subset to start_month and end_month
    ds = select_months_ds(ds, start_mon, end_mon, 'time')
    ds = ds.load()

    ## run students t-test if anomaly == True
    if anomaly == True:    
        popmean = np.zeros([len(ds.latitude), len(ds.longitude)]) ## population mean
        ndays = len(ds.time) # number of unique days
        print(ndays)
        # calculate t-value based on ndays
        a_mean, tval_mask = ttest_1samp_new(a=ds, popmean=popmean, dim='time', n=ndays)

        ## write to netCDF
        out_fname = out_path + '{0}/{2}/filtered_anomaly_composite_{0}_{1}.nc'.format(varname, ssn, region)
        a_mean.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')

        out_fname = out_path + '{0}/{2}/filtered_anomaly_composite_tvals_{0}_{1}.nc'.format(varname, ssn, region)
        tval_mask.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')

    else:
        a_mean = ds.mean('time')
        out_fname = out_path + '{0}/{2}/composite_{0}_{1}.nc'.format(varname, ssn, region)
        a_mean.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')

    return a_mean

In [6]:
%%time
## iterate through options
varname_lst = ['700z', 'ivt']
ssn_lst = ['DJF', 'MAM', 'JJA', 'SON', 'NDJFMA', 'MJJASO']
anom_lst = [True, False]

a = [varname_lst, ssn_lst, anom_lst]

option_lst = list(itertools.product(*a))
for i, lst in enumerate(option_lst):
    anomaly = lst[2]
    ssn = lst[1]
    varname = lst[0]
    print('Anomaly:', anomaly, 'Season:', ssn, 'Variable:', varname)
    tmp = compute_horizontal_composites(varname, anomaly, ar_dates, ssn, region)

Anomaly: True Season: DJF Variable: 700z
1
Anomaly: False Season: DJF Variable: 700z


/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/core/nputils.py:208: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


Anomaly: True Season: MAM Variable: 700z
56
Anomaly: False Season: MAM Variable: 700z
Anomaly: True Season: JJA Variable: 700z
116
Anomaly: False Season: JJA Variable: 700z
Anomaly: True Season: SON Variable: 700z
52
Anomaly: False Season: SON Variable: 700z
Anomaly: True Season: NDJFMA Variable: 700z
21
Anomaly: False Season: NDJFMA Variable: 700z
Anomaly: True Season: MJJASO Variable: 700z
204
Anomaly: False Season: MJJASO Variable: 700z
Anomaly: True Season: DJF Variable: ivt
1
Anomaly: False Season: DJF Variable: ivt


/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/core/nputils.py:208: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/core/nputils.py:208: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/core/nputils.py:208: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


Anomaly: True Season: MAM Variable: ivt
56
Anomaly: False Season: MAM Variable: ivt
Anomaly: True Season: JJA Variable: ivt
116
Anomaly: False Season: JJA Variable: ivt
Anomaly: True Season: SON Variable: ivt
52
Anomaly: False Season: SON Variable: ivt
Anomaly: True Season: NDJFMA Variable: ivt
21
Anomaly: False Season: NDJFMA Variable: ivt
Anomaly: True Season: MJJASO Variable: ivt
204
Anomaly: False Season: MJJASO Variable: ivt
CPU times: user 6.9 s, sys: 42.1 s, total: 49 s
Wall time: 4min 8s
